In [4]:
import sys
import pandas as pd
from numpy import split
import numpy as np

sys.path.append('/Users/yuksehyun/Desktop/ko-flan/task'  )

from base import BaseGenerator
from datasets import load_dataset



In [5]:

dataset = load_dataset("klue",'nli', split="train")

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /Users/yuksehyun/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


In [100]:
df = pd.DataFrame(dataset)

In [101]:
df.head()

,guid,source,premise,hypothesis,label
0,klue-nli-v1_train_00000,NSMC,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,0
1,klue-nli-v1_train_00001,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,2
2,klue-nli-v1_train_00002,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,1
3,klue-nli-v1_train_00003,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,1
4,klue-nli-v1_train_00004,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,2


In [102]:
drop_df = df.drop(df[df['label']==1].index)

In [103]:
drop_df

,guid,source,premise,hypothesis,label
0,klue-nli-v1_train_00000,NSMC,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,0
1,klue-nli-v1_train_00001,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,2
4,klue-nli-v1_train_00004,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,2
6,klue-nli-v1_train_00006,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,0
7,klue-nli-v1_train_00007,wikipedia,10년 만에 찾는 피터를 웬디는 따뜻하게 맞이하고 피터는 성공리에 연설을 마치는데 ...,웬디는 피터를 차갑게 맞이했다.,2
...,...,...,...,...,...
24991,klue-nli-v1_train_24991,NSMC,히틀러와 당시 독일의 상황 속에 들어간 듯 하다.,당시 독일의 상황을 간접적으로 느낀 듯 하다.,0
24992,klue-nli-v1_train_24992,NSMC,히틀러와 당시 독일의 상황 속에 들어간 듯 하다.,당시 독일의 상황 속에 들어간 듯 하다.,0
24993,klue-nli-v1_train_24993,wikitree,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,2
24995,klue-nli-v1_train_24995,wikitree,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,0


In [121]:
new_df = pd.DataFrame()
new_df['input'] = drop_df['premise']

In [122]:
new_df.drop_duplicates(inplace = True)

In [123]:
new_df['postive'] =None
new_df['negative']=None
for i in range(0, new_df.shape[0]):
          row = new_df.iloc[i]
          new_df['postive'].iloc[i] = df.loc[(df['premise']==row['input'])&(df['label']==0),'hypothesis']
          new_df['negative'].iloc[i] = df.loc[(df['premise']==row['input'])&(df['label']==2),'hypothesis']


In [124]:
new_df['postive']=new_df['postive'].apply(lambda x: x[0] if len(x)>0 else None)
new_df['negative']=new_df['negative'].apply(lambda x: x[0] if len(x)>0 else None)

In [145]:
new_df['postive'].isnull().sum()

0

In [146]:
new_df['negative'].isnull().sum()

0

In [143]:
new_df=new_df.dropna(subset=['postive','negative'])

In [147]:
new_df

,input,postive,negative
0,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,힛걸 그 어떤 히어로보다 별로다.
4,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.
7,10년 만에 찾는 피터를 웬디는 따뜻하게 맞이하고 피터는 성공리에 연설을 마치는데 ...,"피터 배닝, 잭, 매기는 남매사이다.",웬디는 피터를 차갑게 맞이했다.
10,"10년 전 한 병원에서 입원 중인 한 소녀가 실종되는 사건이 일어나지만, 목격자도 ...",10년 전 한 병원에서 입원 중인 소녀가 실종된다.,병원에서 실종된 소녀는 경찰들의 수사덕에 하루만에 돌아온다.
13,"10년 전, 공동육아를 매개로 성미산 마을에 들어와 산 강석필, 홍형숙 부부는 맥가...","강석필, 홍형숙 부부가 성미산 마을에 들어온 것은 10년전이다.","강석필, 홍형숙 부부가 성미산 마을에 들어온 이유는 요양이었다."
...,...,...,...
24981,"희수는 병원을 다녀온 엄마의 행동에 이상함을 느끼고 캐묻고, 엄마의 생이 얼마 남지...",엄마는 살 날이 오래 남지는 않았다.,엄마의 병이 호전되었다는 사실에 희수는 기뻐한다.
24984,희영은 아내와 사별하고 아들과 단둘이 살고 있는 성현에게 설렘을 느낀다.,성현의 아내는 죽었다.,성현은 아내와 이혼하고 혼자 살고있다.
24988,히말라야의 바람은 감독의 머리속에서만 머문다,히말라야의 바람은 감독의 머리 안에서만 머문다.,히말라야의 바람은 감독의 머리 속에서 머물지 않았다.
24990,히틀러와 당시 독일의 상황 속에 들어간 듯 하다.,당시 독일의 상황을 간접적으로 느낀 듯 하다.,당시 독일의 상황 속에 들어가지 못한 듯 하다.
